<a href="https://colab.research.google.com/github/ammu993/Data-Analysis-using-python-/blob/main/Data_Analysis_Python_Music_Preferences_Spotify_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# ANALYSIS OF REGIONAL MUSIC PREFERENCES (DEC 2023 - JUN 2024) 🎵 🎹


##PROJECT OVERVIEW

This project analyzes the top Spotify songs in 73 countries over a six-month period from December 1, 2023, to June 1, 2024. The aim is to understand the music preferences of Spotify users across different regions: North America, Latin America, Europe, and the Rest of the World. By examining this dataset, we can identify regional daily trends and top artists that define the global music landscape during this period.

The dataset used in this analysis is [Top Spotify Songs in 73 Countries (Daily Updated)](https://www.kaggle.com/datasets/asaniczka/top-spotify-songs-in-73-countries-daily-updated/data) provided by user **asaniczka** on Kaggle. This dataset is licensed under the Open Data Commons Attribution License (ODC-By) v1.0.

Explanation to the audio features such as  'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature' can be found in Annex section.




---


## ASK


**What were the music preferences across different regions (North America, Latin America, Europe, and the Rest of the World) on Spotify and how did they vary over the period from December 2023 to June 2024?**

--*What were the trends in the popularity and daily rankings of the top songs globally?*

--*How did the daily rankings of the top songs differ across regions?*

--*What regional differences can be observed in the audio characteristics of the most popular songs?*

--*Who were the top artists per region, and how often do their songs appear in the top daily ranks?*



---

## DATA PREPARATION

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.ticker as ticker
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
csv_file_path= "/content/drive/MyDrive/universal_top_spotify_songs.csv"

In [ ]:
df= pd.read_csv(csv_file_path)
pd.set_option('display.max_columns', None)
df.head()

In [ ]:
df.shape

In [ ]:
df.tail()

In [ ]:
df.info()

In [ ]:
df.columns

## DATA PROCESSING


### Data Cleaning


In [ ]:
df.isna().sum()

In [ ]:
#Replacing null values in the country column
df.country=df.country.replace("NaN",np.nan)
df.country=df.country.replace(np.nan,"Global")

In [ ]:
df.isnull().sum()

In [ ]:
#Viewing part of dataset with album_name is NULL
df.loc[df.album_name.isnull()]

In [ ]:
df=df.dropna(subset=['album_name','album_release_date'])

In [ ]:
df.info()

In [ ]:
df.drop_duplicates()

In [ ]:
len(df.country.unique())

In [ ]:
#To get list of country ISO codes mentioned in the data set
array_countries=df['country'].unique()
country_list=array_countries.tolist()


In [ ]:
#Function to allocate regions from country information
def country_to_region(data):
  #Regions dictionary containing values as the ISO codes of countries
  regions_dict={ "Europe": [
        "AT", "BE", "BG", "BY", "CH", "CZ", "DE", "DK", "EE", "ES",
        "FI", "FR", "GB", "GR", "HU", "IE", "IS", "IT", "LT", "LU",
        "LV", "NL", "NO", "PL", "PT", "RO", "SE", "SK", "UA"
    ],
    "North America": [
        "CA", "US"
    ],
    "Latin America": [
        "AR", "BO", "BR", "CL", "CO", "CR", "DO", "EC", "GT", "HN",
        "MX", "NI", "PA", "PE", "PY", "SV", "UY", "VE"
    ],
    "Rest of World": [
        "AE", "AU", "EG", "HK", "ID", "IL", "IN", "JP", "KR", "KZ",
        "MA", "MY", "NG", "NZ", "PH", "PK", "SA", "SG", "TH", "TR",
        "TW", "VN", "ZA"
    ]}
  if data['country'] == 'Global':
    return 'Global'
  elif data['country'] in regions_dict['Europe']:
    return 'Europe'
  elif data['country'] in regions_dict['Rest of World']:
    return 'Rest of World'
  elif data['country'] in regions_dict['Latin America'] :
    return 'LATAM'
  elif data['country'] in regions_dict['North America'] :
    return 'North America'

modified_df = df
modified_df['regions']= modified_df.apply(country_to_region, axis='columns')# btry with replace


In [ ]:
#Deleting the spotify id column
modified_df=modified_df.drop (columns='spotify_id')

In [ ]:
df = modified_df

In [ ]:
#Converting the data type of columns with dates
df['album_release_date'] = pd.to_datetime(df['album_release_date'], format='%Y-%m-%d')
df['snapshot_date'] = pd.to_datetime(df['snapshot_date'], format='%Y-%m-%d')

### EDA

In [ ]:
#Summary statistics for numerical and string data types
df.describe(include=[np.number,object])

In [ ]:
#Summary statistics for numerical and string data types
df.describe(include=np.datetime64)

In [ ]:
#to see correlation between numeric values
corr_matrix= df.corr(numeric_only=True)#Spearman's for relaxed assumption

In [ ]:
custom_palette= ['#26CC50','#7C71BB','#222426']# Hex Color values correspond to green,purple,grey
cmap = LinearSegmentedColormap.from_list("custom", custom_palette, N=256)
sns.heatmap(corr_matrix,annot=True, cmap=cmap)
plt.rcParams['figure.figsize']=(20,20)
plt.show()
plt.tight_layout()

In [ ]:
sns.scatterplot(y=df['energy'],x=df['loudness'],hue=df['is_explicit'],palette=['#26CC50','#222426'])
plt.title('Energy vs Loudness')
plt.figure(figsize=(14,8))
plt.show()
plt.tight_layout()

In [ ]:
sns.scatterplot(y=df['energy'],x=df['acousticness'],hue=df['is_explicit'],palette=['#26CC50','#222426'])
plt.title('Energy vs Acousticness')
plt.figure(figsize=(14,8))
plt.show()
plt.tight_layout()

## ANALYSIS
The main aim  of this section is to answer the research question:

**What were the music preferences across different regions (North America, Latin America, Europe, and the Rest of the World) on Spotify and how did they vary over the period from December 2023 to June 2024?**


In [ ]:
#Filtering for last 6 months of data
df_2024 = df[(df['snapshot_date'] >= '2023-12-01') & (df['snapshot_date'] < '2024-06-01')]

### What are the trends in the popularity and daily rankings of the top songs globally?

*   How does the popularity score trend of the top 5 globally ranked songs compare over the six-month period?
*   How do the daily ranking trends of these top 5 songs change over time?



In [ ]:
#Filtering the data frame to get top song that always ranked between 1-15 globally
top_songs_2024= df_2024.loc[(df_2024.country=='Global') & (df_2024.daily_rank<=15)]
top_songs_2024

In [ ]:
#Selecting top 5 globally trending song which had daily ranks between 1-15
top5dailyrank = top_songs_2024.name.value_counts().head(5)
top5dailyrank=top5dailyrank.reset_index()
top5dailyrank

In [ ]:
songs_array = top5dailyrank['name'].values
songs_top5 = songs_array.tolist()
print(songs_array.dtype)
print(type(songs_top5))
# Convert the array to a list
# songs_top5 = []
# for row in top5dailyrank.index:
#   song= top5dailyrank.iloc[row,0]
#   songs_top5.append(song)
# songs_top5

In [ ]:
#Plotting popularity score and daily rank trend of top 5 songs
colors=['#26CC50','#7C71BB' ,'#A8B0BA','#A8B0BA','#A9B0BA']
linestyles = ['-', '-', '-', '--', ':']
plt.style.use('default')

# Figure with two subplots
fig, (ax1,ax2) = plt.subplots(figsize=(16, 10),nrows=2, ncols=1, sharex=True)

# Plotting each song's popularity score trend
for i, song in enumerate(songs_top5):
     song_data = top_songs_2024[top_songs_2024['name'] == song]
     ax1.plot(song_data['snapshot_date'], song_data['popularity'],color=colors[i],linestyle=linestyles[i], label=song)
# Plotting each song's daily ranking trend
for i, song in enumerate(songs_top5):
    song_data = top_songs_2024[top_songs_2024['name'] == song]
    ax2.plot(song_data['snapshot_date'], song_data['daily_rank'],color=colors[i],linestyle=linestyles[i], label=song)

#Popularity trend
ax1.set_xlabel('Date')
ax1.set_ylabel('Popularity Score')
ax1.set_title('Popularity Score Trend for Top 5 Songs(Global)')
ax1.legend(fontsize='medium',loc='lower right')
ax1.grid(axis= 'x',color='grey', linestyle='-', linewidth=0.5, alpha=0.3)
#Daily rankings
ax2.set_xlabel('Date')
ax2.yaxis.set_major_locator(ticker.MaxNLocator(integer=True))
ax2.set_ylabel('Daily rankings')
ax2.set_title('Daily Global Rankings of Top 5 Songs')
ax2.grid(axis= 'x',color='grey', linestyle='-', linewidth=0.5, alpha=0.3)
plt.tight_layout()


### How do the daily rankings of the top songs differ across regions?

*   What are the daily ranking trends of the top songs in North America, Latin America, Europe, and the Rest of the World?
*   Can seasonal/monthly music preferences across the regions be observed?



In [ ]:
colors=['#26CC50','#7C71BB' ,'#5E6369','#757B82','#A9B0BA']

# Function to create line charts for daily rankings
def plot_daily_rankings(region, top_n):
    # Filter the data for the given region
    regional_data = df_2024[df_2024['regions'] == region]

    # Grouping the data by song name and snapshot_date
    grouped = regional_data.groupby(['name', 'snapshot_date'])['daily_rank'].min().reset_index()

    # Select the top N songs based on the minimum daily rank
    top_songs = grouped['name'].value_counts().head(top_n).index

    # Plotting the daily trend
    fig, ax = plt.subplots(figsize=(14, 6))
    for i,song in enumerate(top_songs):
        song_data = grouped.loc[grouped['name'] == song]
        ax.plot(song_data['snapshot_date'], song_data['daily_rank'],color=colors[i] ,label=song)

    ax.set_title(f'Daily Rankings of Top {top_n} Songs in {region}')
    ax.set_xlabel('Date')
    ax.set_ylabel('Daily Rank')
    ax.legend(loc='upper left')
    plt.show()
    plt.tight_layout()

plot_daily_rankings('Europe',5)
plot_daily_rankings('North America',5)
plot_daily_rankings('LATAM',5)
plot_daily_rankings('Rest of World',5)


###Music preferences based on Song features: What regional differences can be observed in the audio characteristics of the most popular songs?

*   How do audio features (danceability', 'energy', 'key', 'loudness' etc) of top songs compare across different regions?



In [ ]:
#Function to list top n songs within a region based on their minimum daily rankings
def top_music(df1,region, top_n):
    # Filter the data for the given region
    regional_data = df1[df1['regions'] == region]

    # Grouping the data by song name and snapshot_date
    grouped = regional_data.groupby(['name', 'snapshot_date'])['daily_rank'].min().reset_index()

    # Select the top N songs based on the minimum daily rank
    top_songs = grouped['name'].value_counts().head(top_n).reset_index()
    top_songs_list = top_songs['name'].tolist()
    return top_songs_list

#List of audio features (excluding time_signature,mode) including basic song info
song_features=['name','artists','album_release_date','is_explicit', 'key','loudness','danceability', 'energy', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence','tempo']


In [ ]:
#Top 5 songs in Europe alongwith their song features
df_europe = df_2024.loc[df_2024['name'].isin(top_music(df_2024,'Europe', 10))]
top_eu__musicfeatures= df_europe.loc[:,song_features].groupby('name').min().sort_values(by='danceability', ascending=False)
top_eu__musicfeatures
# top_eu__musicfeatures.boxplot(figsize=(20,10))

In [ ]:
df_Latin= df_2024.loc[df_2024['name'].isin(top_music(df_2024,'LATAM', 10))]
top_LATAM_musicfeatures= df_Latin.loc[:,song_features].groupby('name').min().sort_values(by='danceability', ascending=False)
top_LATAM_musicfeatures
# top_LATAM_musicfeatures.boxplot(figsize=(20,10))

In [ ]:
df_NorthAm= df_2024.loc[df_2024['name'].isin(top_music(df_2024,'North America', 10))]
top_NorthAm_musicfeatures= df_NorthAm.loc[:,song_features].groupby('name').min().sort_values(by='danceability', ascending=False)
top_NorthAm_musicfeatures
# top_NorthAm_musicfeatures.boxplot(figsize=(20,10))

In [ ]:
df_rest= df_2024.loc[df_2024['name'].isin(top_music(df_2024,'Rest of World', 10))]
top_rest_musicfeatures= df_rest.loc[:,song_features].groupby('name').min().sort_values(by='danceability', ascending=False)
top_rest_musicfeatures
# top_rest_musicfeatures.boxplot(figsize=(20,10))

In [ ]:
#Plotting mean audio features(that have values between 0-1) of top songs by region
EU_mean_features=top_eu__musicfeatures.describe().loc['mean','danceability':'valence']
LATAM_mean_features=top_LATAM_musicfeatures.describe().loc['mean','danceability':'valence']
NorthAm_mean_features=top_NorthAm_musicfeatures.describe().loc['mean','danceability':'valence']
Rest_mean_features=top_rest_musicfeatures.describe().loc['mean','danceability':'valence']

features = EU_mean_features.index  # Get feature names
regions = ['Europe', 'Latin America', 'North America','Rest of World']
values = [EU_mean_features, LATAM_mean_features, NorthAm_mean_features,Rest_mean_features]

#Bar plot for different features across different regions
colors=['#26CC50','#7C71BB','#5E6369',"#A9B0BA"]
fig, ax = plt.subplots(figsize=(18, 6))
bar_width = 0.25
group_padding = 0.4
num_features = len(features)
num_regions = len(regions)
index = np.arange(len(features))*(num_regions * bar_width + group_padding)# to ensure spacing between the groups

for i, region in enumerate(regions):
    ax.bar(index + i * bar_width, values[i], bar_width, label=region, color=colors[i])

ax.set_xticks(index + (num_regions - 1) * bar_width / 2)
ax.grid(alpha=0.5,linestyle='--')
ax.set_xticklabels(features, rotation=45, ha='right')
ax.set_title('Mean Song Features of Top Songs by Region', fontsize=12)
ax.legend(loc='upper right')
plt.tight_layout()
plt.show()

In [ ]:
#Plotting mean tempo of top songs by region
tempo_NorthAm=top_NorthAm_musicfeatures.describe().loc['mean','tempo']
tempo_EU=top_eu__musicfeatures.describe().loc['mean','tempo']
tempo_LATAM=top_LATAM_musicfeatures.describe().loc['mean','tempo']
tempo_Rest=top_rest_musicfeatures.describe().loc['mean','tempo']

tempo_values = [tempo_EU,tempo_LATAM, tempo_NorthAm,tempo_Rest]


fig, ax = plt.subplots(figsize=(10, 6))

# Bar plot
regions = ['Europe', 'Latin America', 'North America','Rest of World']
colors=['#26CC50','#7C71BB','#5E6369',"#A9B0BA"]
bar_width = 0.5
index = np.arange(len(regions))
bars = ax.bar(index, tempo_values, bar_width, color=colors)


ax.set_xlabel('Region')
ax.set_ylabel('Mean Tempo (BPM)')
ax.set_title('Mean Tempo of Top Songs by Region', fontsize=12)
ax.set_xticks(index)
ax.set_xticklabels(regions)
plt.tight_layout()



In [ ]:
#Plotting mean loudness of top songs by region
loud_NorthAm=top_NorthAm_musicfeatures.describe().loc['mean','loudness']
loud_EU=top_eu__musicfeatures.describe().loc['mean','loudness']
loud_LATAM=top_LATAM_musicfeatures.describe().loc['mean','loudness']
loud_Rest=top_rest_musicfeatures.describe().loc['mean','loudness']

loud_values = [loud_EU, loud_LATAM, loud_NorthAm, loud_Rest]

# Set up the figure and axis
fig, ax = plt.subplots(figsize=(10, 6))

# Bar plot
regions = ['Europe', 'Latin America', 'North America','Rest of World']
colors=['#26CC50','#7C71BB','#5E6369',"#A9B0BA"]
bar_width = 0.5
index = np.arange(len(regions))
bars = ax.bar(index, loud_values, bar_width, color=colors)

ax.set_xlabel('Region')
ax.set_ylabel('Mean Loudness(-60db to 0)')
ax.set_title('Mean Loudness of Top Songs by Region', fontsize=12)
ax.set_xticks(index)
ax.set_xticklabels(regions)
plt.tight_layout()


### Music preference based on Artists : Who are the top artists per region, and how often do their songs appear in the top daily ranks?

*   Which artists have the most songs appearing in the top 10 daily ranks in each region?


In [ ]:
# Top_Region_Artists are artists whose songs appeared multiple times in the top 50 daily ranks for the specific region, considering only artists whose songs have occupied one of the top 10 positions in the daily ranks consistently.
top_eu_artists= df_2024.artists.loc[(df_2024.regions=='Europe')& (df_2024.daily_rank<=10)].value_counts().head(5)
top_latam_artists= df_2024.artists.loc[(df_2024.regions=='LATAM')& (df_2024.daily_rank<=10)].value_counts().head(5)
top_NorthAm_artists= df_2024.artists.loc[(df_2024.regions=='North America')& (df_2024.daily_rank<=10)].value_counts().head(5)
top_rest_artists= df_2024.artists.loc[(df_2024.regions=='Rest of World')& (df_2024.daily_rank<=10)].value_counts().head(5)

#To sort the appearance of bars in descending order
top_eu_artists=top_eu_artists.iloc[::-1]
top_latam_artists=top_latam_artists.iloc[::-1]
top_NorthAm_artists=top_NorthAm_artists.iloc[::-1]
top_rest_artists=top_rest_artists.iloc[::-1]

#Plotting horizontal bar charts for top 5 artists across regions
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(nrows=2,ncols=2,figsize=(14, 6))
plt.suptitle('Top 5 Artists across Regions', fontsize=20)
ax1.barh(top_eu_artists.index,top_eu_artists.values,color="#26CC50")
ax1.set_title('Europe')
ax2.barh(top_latam_artists.index,top_latam_artists.values,color="#7C71BB")
ax2.set_title('LATAM')
ax3.barh(top_NorthAm_artists.index,top_NorthAm_artists.values,color="#5E6369")
ax3.set_title('North America')
ax4.barh(top_rest_artists.index,top_rest_artists.values,color="#A9B0BA")
ax4.set_title('Rest of World')
plt.tight_layout()


In [ ]:
#To see the songs that made the artist top the region

top_eu= df_2024.loc[(df_2024.regions=='Europe')& (df_2024.daily_rank<=10)]
top_eu_grouped=top_eu.groupby(['artists','name']).size().reset_index(name='counts')
top_eu_grouped.sort_values(by= 'counts',ascending=False)

# top_latam= df_2024.loc[(df_2024.regions=='LATAM')& (df_2024.daily_rank<=10)]
# top_latam_grouped= top_latam.groupby(['artists','name']).size().reset_index(name='counts')
# top_latam_grouped.sort_values(by= 'counts',ascending=False)

# top_NorthAm= df_2024.loc[(df_2024.regions=='North America')& (df_2024.daily_rank<=20)]
# top_NorthAm_grouped=top_NorthAm.groupby(['artists','name']).size().reset_index(name='counts')
# top_NorthAm_grouped.sort_values(by= 'counts',ascending=False)

# top_rest= df_2024.loc[(df_2024.regions=='Rest of World')& (df_2024.daily_rank<=10)]
# top_rest_grouped=top_rest.groupby(['artists','name']).size().reset_index(name='counts')
# top_rest_grouped.sort_values(by= 'counts',ascending=False)



## RESULTS

Analysis conducted to determine **"What were the music preferences across different regions (North America, Latin America, Europe, and the Rest of the World) on Spotify and how did they vary over the period from December 2023 to June 2024?"** resulted in the following findings:


- Daily rankings gives a ***better measurement of trend of music preference*** globally rather than popularity score as songs with 0 popularity score can be ranking 1.
- Daily ranking trends of top 5 songs in different regions show the following patterns:
  - In Europe, top songs that ranked high(1-10) in the winter months moved further down the ranking list after April.
  - In North American, top songs ranked low(>10) in December and between Jan and mid April held a consistent high rankings.
  - In Latin America and Rest of World regions,the rankings exhibit significant fluctuations.

- Music preferences according to audio features of most popular songs in each region:
  - **Danceable songs** are preferred highly in all regions especially in Europe and Latin America.
  - The **energy** levels are fairly similar across all regions, with Europe and Latin America showing slightly higher values.
  - Europe shows a slightly higher value of **speechiness** in comparison to other regions, indicating a slight preference for tracks with more spoken words.
  - Europe shows a slightly higher preference for **acoustic and instrumental** tracks.
  - North America and the Rest of the World show slightly higher values, indicating a minor preference for **live** performance elements in music.
  - Mean **valence** is highest in Latin America followed by rest of world and Europe, which indicates preference of positive or euphoric songs in these regions.
  - Latin America and North America prefer **faster-paced** music, while Europe favors slower-paced tracks. The Rest of the World falls in between but leans towards faster tempos.
  - Latin America prefers the **loudest** music, followed by North America and then Europe. The Rest of the World prefers the quietest tracks.

- Top artists in each region whose songs ranked consistently less than 10 over the last 6 months:
  - Benson Boone(Europe, 1847 times)
  - Xavi(LATAM, 2458)
  - Noah Kahan(North America, 231)
  - Jimin(Rest of World, 1227)






## NEXT ACTIONS



Based on above finding following actions can taken by :-

1.   Artists and Record Labels:

  - Tailored Releases: Customize music releases to match regional preferences
      - Latin America: Focus on energetic, upbeat tracks.
      - Europe: Emphasize danceable tracks with acoustic and instrumental elements.
      - North America: Balance energetic tracks with live performance appeal.
  - Timing of Releases: Plan major releases strategically
      - Europe: Before April to maintain high rankings.
      - North America: Between January and mid-April for stability.
  - Targeted Collaborations: Collaborate with local top artists to enhance market penetration.



2.   Spotify:
  - Algorithm Adjustments: Refine recommendation algorithms to reflect regional music tastes, focusing on tempo, loudness, and danceability.

  - Regional Playlists: Curate playlists that resonate with specific regional preferences, enhancing user engagement.

  - Seasonal Campaigns:Develop seasonal marketing campaigns aligned with regional music trends to optimize user interaction and satisfaction.

  - Conduct deeper analysis on why older songs are trending to improve recommendation accuracy and user engagement.

## ANNEX:*Dataset Columns and Parameters*

Detailed documentation in [Spotify for developers](https://developer.spotify.com/documentation/web-api/reference/get-audio-features)

**Track Attributes**

- **name**: The name of the track or song.
- **artist**: The name of the artist who performed the song.
- **album**: The name of the album the song belongs to.
- **duration_ms**: The duration of the song in milliseconds.
- **popularity**: A value between 0 and 100 that represents the track's popularity globally on Spotify, with higher values indicating greater popularity.

**Audio** **Features**

- **danceability**: A value between 0 and 1 that describes how suitable the track is for dancing, based on a combination of musical elements like tempo, rhythm stability, and beat strength.
- **energy**: A measure between 0 and 1 that represents the perceived intensity and activity of the track, with higher values indicating more energetic tracks.
- **key**: The estimated key that the track is in (e.g., 0 = C, 1 = C♯/D♭, 2 = D, etc.).
- **loudness**: The overall loudness of the track in decibels (dB), with higher values indicating louder tracks.
- **mode**: Indicates the modality (major or minor) of the track, with 0 = minor and 1 = major.
- **speechiness**: A value between 0 and 1 that detects the presence of spoken words in the track, with higher values indicating more spoken words.
- **acousticness**: A confidence measure between 0 and 1 of whether the track is acoustic, with higher values indicating a more acoustic sound.
- **instrumentalness**: A value between 0 and 1 that represents the likelihood that the track contains no vocals, with higher values indicating an instrumental track.
- **liveness**: A measure between 0 and 1 that detects the presence of an audience in the recording, with higher values indicating a live performance.
- **valence**: A measure between 0 and 1 that describes the musical positiveness conveyed by the track, with higher values indicating more positive or euphoric emotions.
- **tempo**: The overall estimated tempo of the track in beats per minute (BPM).
- **time_signature** : The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). The time signature ranges from 3 to 7 indicating time signatures of "3/4", to "7/4".